Load data from drive

In [ ]:
from google.colab import drive
import import_ipynb
import sys

drive.mount('/content/drive/',force_remount = False)
proj_dir_path = '/content/drive/MyDrive/Study_materials/Voice disorder detection project/'
sys.path.append(proj_dir_path)
%cd $proj_dir_path


my_voice_data_no_spaces = '/content/drive/MyDrive/Study_materials/Voice disorder detection project/patients'
my_voice_data = '/content/drive/MyDrive/Study_materials/Voice\ disorder\ detection\ project/patients'

imports and installations


In [ ]:
# install google's VAD
!pip install webrtcvad

#install librosa and sox
!apt-get install libsox-fmt-all
!apt-get install sox
!pip install sox
!pip install dill

In [ ]:
!pip install -- dill

In [ ]:

## Imports

import os
import glob
import numpy as np
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
from IPython.display import Audio
import pandas as pd
from scipy.io import wavfile
import re
import dill
# from sox.file_info import sample_rate
from datetime import date
import shutil


import webrtcvad
import sox

print(torch.__version__)
print(torchaudio.__version__)

## Use cuda if available
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

2.2.1+cu121
2.2.1+cu121
cpu


Constants and helper func definitions

In [ ]:
# Define constants
wanted_sample_rate = 16000

audio_labels_compact = ["E","LE","HE",
                "Ah","LAh","HAh",
                "Ooh","LOoh","HOoh",
                "Mmm", "Sen","Diag"]
audio_labels = ["normal pitch E","low pitch E","high pitch E",
                "normal pitch Ah","low pitch Ah","high pitch Ah",
                "normal pitch Ooh","low pitch Ooh","high pitch Ooh",
                "normal pitch Mmmmm","Sentence","Diagnosis"]

def get_num_of_chunk_as_int(chunk):
  # receives a chunk's name in str and extracts the num as integer
  # expects the chunk name to be in the a specific format
  return (chunk.split(".")[0]).split("-")[1]

class Chunk_obj:
  # patient_num - Patient's number
  # chunk_num - patient's audio chunk number (chronological)
  # sample_rate - chunk's sample rate
  # data - The chunk's recording data - np array
  def __init__(self, patient_num,chunk_num, sample_rate,data):
    self.patient_num = patient_num
    if (chunk_num <= 9):
      self.label = audio_labels_compact[chunk_num]
    else:
      self.label = audio_labels[10]
    self.sample_rate = sample_rate
    self.data = data

  def get_data(self):
    return self.data

  def __str__(self):
    return f" P-{self.patient_num} {self.label} "

Google's VAD wrapper methods

In [ ]:
import collections
import contextlib
import sys
import wave

def read_wave(path):
    """Reads a .wav file.

    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.

    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)



class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.

    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.

    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.

    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.

    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.

    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.

    Arguments:

    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).

    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])

def check_chunk_files():
  chunk_list = glob.glob('/content/chunk-??.wav')
  if len(chunk_list) > 0:
    return True
  else:
    return False

Google VAD usage method

In [ ]:
def webrtcvad_usage(aggressive=0, audio_path=None,frame_duration=30,padding_duration=300,min_chunk_length = 0.5):
    if os.path.isfile('/content/chunk-00.wav'):
      print("Clearing previous .wav files")
      !rm chunk-??.wav
      !rm chunk-???.wav
    vad = webrtcvad.Vad(aggressive)
    audio, sample_rate  = read_wave(audio_path)
    frames = frame_generator(frame_duration, audio, sample_rate)
    frames = list(frames)
    segments = vad_collector(sample_rate, frame_duration, padding_duration, vad, frames)



  #  sample_rate - The audio sample rate, in Hz.
  #   frame_duration_ms - The frame duration in milliseconds.
  #   padding_duration_ms - The amount to pad the window, in milliseconds.
  #   vad - An instance of webrtcvad.Vad.
  #   frames - a source of audio frames (sequence or generator).
    i = 0
    for j,segment in enumerate(segments):
      path = 'chunk-%002d.wav' % (i,)
      # print(' Writing %s' % (path,))
      len_sec = len(segment)/(sample_rate*2)
      # if( i < 10):
      if((len_sec >= min_chunk_length) ):
        write_wave(path, segment, sample_rate)
        # print(f'\n phone Chunk {i} length is {len_sec} seconds')
        i = i+1
      # else:
        # print(f'\n too short! Chunk {i} length is {len_sec} seconds')
    return segments

Given a patient number, run it through sox, resampling to 16k and converting to mono-channel

# Phase 1
each vowel gets it's own label

# Phase 2
Cleaning noise from data, and splitting the cleaned .wav file into

In [ ]:
#Phase 1
def audio_split_phase1(pat_num = -1):
  if(pat_num == -1):
    print("No pat num or df given")
    return

    ## Sox usage - resample and monochannel
  # file name can start with capital P or lowercase p, we take the monochannel
  !sox {my_voice_data}/Patient\ {pat_num}.wav -c 1 pat_{pat_num}_sox.wav
  !sox {my_voice_data}/patient\ {pat_num}.wav -c 1 pat_{pat_num}_sox.wav
  # resample to 16k
  !sox /content/pat_{pat_num}_sox.wav -r 16000 /content/pat_{pat_num}_sox_mono.wav
  # delete older file
  !rm pat_{pat_num}_sox.wav

  # google VAD usage
  path = 'pat_'+ str(pat_num)+'_sox_mono.wav'
  webrtcvad_usage(aggressive=3, audio_path = path,frame_duration=10,padding_duration=150,min_chunk_length = 0.90)
  chunk_list = [None] * 12
  cbn=sox.Combiner()
  sounds=[]
  for chunk in glob.glob('/content/chunk-??.wav'): ## we assume there are less than 100 chunks
    chunk_num = int(get_num_of_chunk_as_int(chunk))
    if(chunk_num > 9): # chunk 10+ are being merged into a single sentence
      sounds.append(chunk)
    else:                                        # chunk 0-9 are being labled into df
      sample_rate, temp_wav = wavfile.read(chunk)
      print("sample_rate is ",sample_rate)
      chunk_obj = Chunk_obj(patient_num=pat_num,chunk_num =chunk_num,sample_rate=sample_rate,data=temp_wav)
      chunk_list[chunk_num] = chunk_obj

  #PROCESS SOUND PATHS TO AN ARRAY
  if len(sounds)>=2:
      print(sounds)
      cbn.build(sounds,'sentence.wav','concatenate')

  # insert sentence.wav into df
  # Create chunkObj
  try:
    _, temp_wav_sentence = wavfile.read('/content/sentence.wav')
    sentence_chunk_obj = Chunk_obj(patient_num=pat_num,chunk_num = 11,sample_rate=sample_rate,data=temp_wav_sentence)
    chunk_list[10] = sentence_chunk_obj
  except:
    chunk_list[10] = "No sentence audio"



  !rm pat_{pat_num}_sox_mono.wav
  !rm sentence.wav
  return chunk_list

def audio_split_phase2(pat_num = -1,aggressive=3,frame_duration=10,padding_duration=150,save_to_drive= False):
  if(pat_num == -1):
    print("No pat num or df given")
    return

    ## Sox usage - resample and monochannel
  # file name can start with capital P or lowercase p, we take the monochannel
  !sox {my_voice_data}/Patient\ {pat_num}.wav -c 1 pat_{pat_num}_sox.wav
  !sox {my_voice_data}/patient\ {pat_num}.wav -c 1 pat_{pat_num}_sox.wav
  # resample to 16k
  !sox /content/pat_{pat_num}_sox.wav -r 16000 /content/pat_{pat_num}_sox_mono.wav
  # delete older file
  !rm pat_{pat_num}_sox.wav

  # google VAD usage
  path = 'pat_'+str(pat_num)+'_sox_mono.wav'
  webrtcvad_usage(aggressive=3, audio_path=path,frame_duration=10,padding_duration=150,min_chunk_length = 0.90)
  chunk_list = [None] * 100
  cbn=sox.Combiner()
  if(check_chunk_files()):
    for chunk in glob.glob('/content/chunk-??.wav'): ## we assume there are less than 100 chunks
      chunk_num = int(get_num_of_chunk_as_int(chunk))
      chunk_list[chunk_num] = chunk
  else:
    print(f"Recording {pat_num} is too noisy to be split")
    return [None]


  # Combine all chunks into a single file
  cbn=sox.Combiner()
  chunk_list = [x for x in chunk_list if x is not None]
  if len(chunk_list)>=1:
      print(chunk_list)
      cbn.build(chunk_list,'test.wav','concatenate')

  try:
    if(save_to_drive):
      source_path = '/content/test.wav'
      name = "patient_" + str(pat_num)+".wav"
      destination_path = '/content/drive/MyDrive/Study_materials/Voice disorder detection project/patients_preproccessed/'+name
      # Move and rename the file
      shutil.move(source_path, destination_path)
    else:
      sample_rate, temp_wav_sentence = wavfile.read('/content/test.wav')
      sentence_chunk_obj = Chunk_obj(patient_num=pat_num,chunk_num = 10,sample_rate=sample_rate,data=temp_wav_sentence)
  except Exception as e:
    print("Something happened:", e)



  !rm pat_{pat_num}_sox_mono.wav
  !rm test.wav
  !rm chunk-**.wav
  !rm chunk-***.wav
  return


def audio_split_phase3(pat_num = -1,aggressive=3,frame_duration=10,padding_duration=150,save_to_drive= False):
  if(pat_num == -1):
    print("No pat num or df given")
    return

    ## Sox usage - resample and monochannel
  # file name can start with capital P or lowercase p, we take the monochannel
  !sox {my_voice_data}/Patient\ {pat_num}.wav -c 1 pat_{pat_num}_sox.wav
  !sox {my_voice_data}/patient\ {pat_num}.wav -c 1 pat_{pat_num}_sox.wav
  # resample to 16k
  !sox /content/pat_{pat_num}_sox.wav -r 16000 /content/pat_{pat_num}_sox_mono.wav
  # delete older file
  !rm pat_{pat_num}_sox.wav

  # google VAD usage
  path = 'pat_'+str(pat_num)+'_sox_mono.wav'
  webrtcvad_usage(aggressive=3, audio_path=path,frame_duration=10,padding_duration=150,min_chunk_length = 0.90)
  chunk_list = [None] * 100
  cbn=sox.Combiner()
  if(check_chunk_files()):
    for chunk in glob.glob('/content/chunk-??.wav'): ## we assume there are less than 100 chunks
      chunk_num = int(get_num_of_chunk_as_int(chunk))
      chunk_list[chunk_num] = chunk
  else:
    print(f"Recording {pat_num} is too noisy to be split")
    return [None]


  # Combine all chunks into a single file
  cbn=sox.Combiner()
  chunk_list = [x for x in chunk_list if x is not None]
  if len(chunk_list)>=1:
      print(chunk_list)
      cbn.build(chunk_list,'test.wav','concatenate')

  try:
    if(save_to_drive):
      source_path = '/content/test.wav'
      name = "patient_" + str(pat_num)+".wav"
      destination_path = '/content/drive/MyDrive/Study_materials/Voice disorder detection project/patients_snippets/'+name
      # Move and rename the file
      shutil.move(source_path, destination_path)
    else:
      sample_rate, temp_wav_sentence = wavfile.read('/content/test.wav')
      sentence_chunk_obj = Chunk_obj(patient_num=pat_num,chunk_num = 10,sample_rate=sample_rate,data=temp_wav_sentence)
  except Exception as e:
    print("Something happened:", e)



  !rm pat_{pat_num}_sox_mono.wav
  !rm test.wav
  !rm chunk-**.wav
  !rm chunk-***.wav
  return

In [ ]:
# prompt: split an audio file into X equal length files using sox

!sox input.wav output%03d.wav trim 0 5


In [ ]:
audio_split_phase2(pat_num=15,save_to_drive=True)

In [ ]:
!pip install xlrd

In [ ]:
file_path = '/content/drive/MyDrive/Study_materials/Voice disorder detection project/patients/CRF table.xlsx'
patients_sheet = pd.read_excel(file_path)

In [ ]:
# Remove all rows that do not hold any data
patients_sheet = patients_sheet[patients_sheet['Age'].notna()]

In [ ]:
num_of_patients_recorded = len(patients_sheet)

In [ ]:
all_patients_diagnosis_col = [-1] * (num_of_patients_recorded+1)

In [ ]:
# Create a bin list of known diseases
list_of_diseases = []
for index, row in patients_sheet.iterrows():
  print("INDEX+1:",index+1," diag: ",row['dysphonia diagnosis'])
  diagnosis = row['dysphonia diagnosis']
  if type(diagnosis) == str:
    for word in diagnosis.split("+"):
      stripped = word.strip()
      if stripped not in list_of_diseases:
        list_of_diseases.append(stripped)

for index, row in patients_sheet.iterrows():
  diagnosis = row['dysphonia diagnosis']
  if type(diagnosis) == str:
    diagnosis_list = row['dysphonia diagnosis'].split("+")
    stripped_list = []
    for diagnosis in diagnosis_list:
      stripped_list.append(diagnosis.strip())
      all_patients_diagnosis_col[index+1] = stripped_list

Our "bank" of seen diseases:

In [ ]:
# list_of_diseases

In [ ]:
# all_patients_diagnosis_col

Main loop: Iterate over all patients data in drive

In [ ]:
# create empty list of lists,
pat_list = [[None]] * (num_of_patients_recorded+1)
# Iterate over all wav files in the drive
def iterate_all_pats(phase = 1):
  for file in glob.glob(my_voice_data_no_spaces + "/*.wav"):
      pat_num_str = re.findall(r'\d+', file)[0]
      print(f'------------ Working on patient {pat_num_str} -------------- ')
      if( int(pat_num_str) <= num_of_patients_recorded):
        if(phase == 1):
          pat_list[int(pat_num_str)] = audio_split_phase1(pat_num_str)
        if(phase == 2):
          pat_list[int(pat_num_str)] = audio_split_phase2(pat_num_str,save_to_drive=True)
      else:
        print("Patient is not on the excel table")

  # Convert list of lists into pandas df
  if(phase == 1):
    df = pd.DataFrame(pat_list, columns = audio_labels)
  if(phase == 2):
    df = pd.DataFrame(pat_list, columns = ["Audio"])
  # Adds diagnosis columns from excel sheet
  df['Diagnosis'] = all_patients_diagnosis_col
  return df

# choose
def create_audio_df(phase = 0):
  if(phase==0):
    print("No phase chosen, 1 for vowel chunks, 2 for full recordings minus the noise")
  else:
    df = iterate_all_pats(phase = phase)
    # create df file name with path
    today = date.today()
    print(today)
    csv_save_path = '/content/drive/MyDrive/Study_materials/Voice disorder detection project/patients_df_'+ str(today)
    # Save the file
    dill.dump(df, file = open(csv_save_path+".pickle", "wb"))
    return df


In [ ]:
df_created = create_audio_df(phase = 2)

In [ ]:
# Reload the file
load = '/content/drive/MyDrive/Study_materials/Voice disorder detection project/patients_df_2024-04-21.pickle'
df_reloaded = dill.load(open(load, "rb"))

In [ ]:
df_reloaded[235:245]

Example on how to access data

In [ ]:
# Accessing patient 2, index 0 recording ( low pitch E)

# patient_num - Patient's number
# chunk_num - patient's audio chunk number (chronological)
# sample_rate - chunk's sample rate
# date - The chunk's recording data - np array
df_reloaded.iloc[127,0].data

array([-367, -303, -226, ..., -180, -157, -133], dtype=int16)

In [ ]:
a = df_reloaded.iloc[127,9 ]
Audio(a.data, rate=16000)